# Wielowarstwowa sieć neuronowa

(*Multilayer perceptron*, *feedforward neural network*)



**Uwaga:** "Input layer" pomimo tego, że ma w nazwie słowo "warstwa", to tak naprawdę to nie jest żadna warstwa sieci... To są po prostu dane wejściowe... Niestety przyjęło się literaturze nazywanie tego w ten sposób, co jest mylące :(


Sieci uczy sie metodą spadku gradientu (pewnymi wariantami tej metody). Uczenie wykorzystuje algorytm **propagacji wstecznej** (https://en.wikipedia.org/wiki/Backpropagation).

<br>

<br>

<br>

**Uwaga!** Sieci neuronowe absolutnie zawsze wymagają zestandaryzowanych danych! Niezależnie od tego czy wykorzystujemy regularyzację czy nie i niezależnie od typu sieci!

<br>

<br>

### Fakt matematyczny: jednowarstwową siecią możemy otrzymać dowolny kształt. 

Co z tego wynika? To, że (teoretycznie) zawsze wystarczy sieć jednowarstwowa (odpowiednio duża). W praktyce rzeczywiście z reguły wystarcza jedna warstwa, ale mimo wszystko zawsze warto sprawdzić czy 2 (lub 3) nie zadziałają przypadkiem lepiej. Przy czym jeżeli dla dwóch wartsw jest gorzej, to nie ma sensu sprawdzać dla większej ilości.

In [7]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score


from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

import warnings
warnings.filterwarnings("ignore")

# Zad
* Wczytaj zbiór danych
* Podziel dane na train test
* Wykonaj uczenie modeli (dobierz najlepsze parametry)
    * LogisticRegression
    * LinearSVC
    * SVC
    * KNeighborsClassifier
    * DecisionTreeClassifier
    * RandomForestClassifier
    * BaggingClassifier
    * ExtraTreesClassifier
    * AdaBoostClassifier
    * GradientBoostingClassifier
    * VotingClassifier
    * xgboost.XGBClassifier
* Porównaj wyniki na zbiorze uczącym    

In [8]:
dataset = np.loadtxt('Dane/diabetes.csv', delimiter=",", skiprows=1)

X = dataset[:,0:8]
Y = dataset[:,8]

print(X.shape)
print(np.mean(Y))

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

from sklearn.model_selection import StratifiedKFold

seed=123
kfold = StratifiedKFold(n_splits=5, random_state=None)

(768, 8)
0.3489583333333333


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', LogisticRegression(C=1))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_1.fit(X_train, y_train)
grid_1.best_params_

{'classifier__C': 1, 'preprocessing': None}

In [46]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', LinearSVC(C=1))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid_2 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_2.fit(X_train, y_train)
grid_2.best_params_

{'classifier__C': 0.1, 'preprocessing': StandardScaler()}

In [32]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC(kernel="rbf", probability=True))])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
            'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
}

grid_3 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_3.fit(X_train, y_train)
grid_3.best_params_

{'classifier__C': 10,
 'classifier__gamma': 0.01,
 'preprocessing': StandardScaler()}

In [16]:
from sklearn.neighbors import KNeighborsClassifier

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', KNeighborsClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__n_neighbors': [1, 3, 5, 10],
            'classifier__leaf_size': [10, 20, 30, 50],
}

grid_4 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_4.fit(X_train, y_train)
grid_4.best_params_

{'classifier__leaf_size': 10,
 'classifier__n_neighbors': 10,
 'preprocessing': None}

In [17]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', DecisionTreeClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__max_depth': [1, 3, 5, 10, 20, 30],
}

grid_5 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_5.fit(X_train, y_train)
grid_5.best_params_

{'classifier__max_depth': 3, 'preprocessing': StandardScaler()}

In [18]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', RandomForestClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__max_depth': [1, 3, 5, 10, 20, 30],
            'classifier__n_estimators': [1, 3, 5, 10, 20],
}

grid_6 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_6.fit(X_train, y_train)
grid_6.best_params_

{'classifier__max_depth': 5,
 'classifier__n_estimators': 10,
 'preprocessing': StandardScaler()}

In [19]:
from sklearn.ensemble import BaggingClassifier
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', BaggingClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__n_estimators': [1, 3, 5, 10, 20],
            'classifier__max_samples': [0.1, 0.3, 0.5, 0.9],
}

grid_7 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_7.fit(X_train, y_train)
grid_7.best_params_

{'classifier__max_samples': 0.5,
 'classifier__n_estimators': 10,
 'preprocessing': None}

In [20]:
from sklearn.ensemble import ExtraTreesClassifier
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', ExtraTreesClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__n_estimators': [1, 3, 5, 10, 20],
            'classifier__max_depth': [2, 5, 10, 15, 20],
}

grid_8 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_8.fit(X_train, y_train)
grid_8.best_params_

{'classifier__max_depth': 10,
 'classifier__n_estimators': 10,
 'preprocessing': None}

In [21]:
from sklearn.ensemble import AdaBoostClassifier
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', AdaBoostClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__n_estimators': [1, 3, 5, 10, 20],
            'classifier__learning_rate': [0.001, 0.01, 0.1, 0.5],
}

grid_9 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_9.fit(X_train, y_train)
grid_9.best_params_

{'classifier__learning_rate': 0.5,
 'classifier__n_estimators': 10,
 'preprocessing': StandardScaler()}

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', GradientBoostingClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__n_estimators': [1, 3, 5, 10, 20],
            'classifier__learning_rate': [0.001, 0.01, 0.1, 0.5],
}

grid_10 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_10.fit(X_train, y_train)
grid_10.best_params_

{'classifier__learning_rate': 0.5,
 'classifier__n_estimators': 3,
 'preprocessing': StandardScaler()}

In [39]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier([('lr', LogisticRegression()), ('svc', SVC(kernel="rbf", probability=True)), ('dt', DecisionTreeClassifier())], voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('svc', SVC(probability=True)),
                             ('dt', DecisionTreeClassifier())],
                 voting='soft')

In [30]:
from xgboost import XGBClassifier
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', XGBClassifier())])

param_grid = {
    'preprocessing': [StandardScaler(), None],
    'classifier__max_depth': [5, 8],
    'classifier__learning_rate': [0.01, 0.05],
    'classifier__n_estimators': [100, 150],
    'classifier__gamma': [0.5, 1],
    'classifier__colsample_bytree': [0.8, 0.5],
    'classifier__subsample': [0.8, 0.5],
    'classifier__min_child_weight': [1, 5]
}

grid_11 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True, verbose=1)

grid_11.fit(X_train, y_train)
grid_11.best_params_

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


{'classifier__colsample_bytree': 0.5,
 'classifier__gamma': 0.5,
 'classifier__learning_rate': 0.01,
 'classifier__max_depth': 5,
 'classifier__min_child_weight': 1,
 'classifier__n_estimators': 150,
 'classifier__subsample': 0.5,
 'preprocessing': StandardScaler()}

In [40]:
from sklearn import  metrics


models = []
models.append(('LR', grid_1.best_estimator_))
models.append(('SVM linear', grid_2.best_estimator_))
models.append(('SVM RBF', grid_3.best_estimator_))
models.append(('KNN', grid_4.best_estimator_))
models.append(('DecisionTreeClassifier', grid_5.best_estimator_))
models.append(('BaggingClassifier', grid_6.best_estimator_))
models.append(('RandomForestClassifier', grid_7.best_estimator_))
models.append(('ExtraTreesClassifier', grid_8.best_estimator_))
models.append(('AdaBoostClassifier', grid_9.best_estimator_))
models.append(('GradientBoostingClassifier', grid_10.best_estimator_))
models.append(('XGBClassifier', grid_11.best_estimator_))
models.append(('voting_clf', voting_clf))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    
    if (name == 'SVM linear'):
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.decision_function(X_test)) ))            
    else:
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))
    if (name == 'SVM linear'):
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.decision_function(X_test)))        
    else:    
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))

LR
precision_score: 0.7435897435897436
recall_score: 0.6304347826086957
f1_score: 0.6823529411764706
accuracy_score: 0.7874015748031497
roc_auc_score: 0.8311191626409018
SVM linear
precision_score: 0.7341772151898734
recall_score: 0.6304347826086957
f1_score: 0.6783625730994152
accuracy_score: 0.7834645669291339
roc_auc_score: 0.829777241009125
SVM RBF
precision_score: 0.7236842105263158
recall_score: 0.5978260869565217
f1_score: 0.6547619047619048
accuracy_score: 0.7716535433070866
roc_auc_score: 0.8391706924315621
KNN
precision_score: 0.7121212121212122
recall_score: 0.5108695652173914
f1_score: 0.5949367088607596
accuracy_score: 0.7480314960629921
roc_auc_score: 0.7864667203435319
DecisionTreeClassifier
precision_score: 0.6666666666666666
recall_score: 0.5652173913043478
f1_score: 0.6117647058823529
accuracy_score: 0.7401574803149606
roc_auc_score: 0.8119296833064948
BaggingClassifier
precision_score: 0.7037037037037037
recall_score: 0.6195652173913043
f1_score: 0.6589595375722543
a

In [42]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['LR','DVM Linear','SVM rbf','KNN', 'DecisionTreeClassifier','BaggingClassifier','RandomForestClassifier','ExtraTreesClassifier', 'AdaBoostClassifier','GradientBoostingClassifier','XGBClassifier', 'voting'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
0,LR,0.743590,0.630435,0.682353,0.787402,0.831119
1,DVM Linear,0.734177,0.630435,0.678363,0.783465,0.829777
2,SVM rbf,0.723684,0.597826,0.654762,0.771654,0.839171
3,KNN,0.712121,0.510870,0.594937,0.748031,0.786467
4,DecisionTreeClassifier,0.666667,0.565217,0.611765,0.740157,0.811930
5,BaggingClassifier,0.703704,0.619565,0.658960,0.767717,0.831186
6,RandomForestClassifier,0.670732,0.597826,0.632184,0.748031,0.817666
7,ExtraTreesClassifier,0.671642,0.489130,0.566038,0.728346,0.800959
8,AdaBoostClassifier,0.686747,0.619565,0.651429,0.759843,0.813137
9,GradientBoostingClassifier,0.677778,0.663043,0.670330,0.763780,0.816928


# MLPClassifier

Dodajmy model sieci neuronowej

In [43]:
from sklearn.neural_network import MLPClassifier

In [44]:
model = MLPClassifier((20,10))
model.fit(X_train, y_train)

y_pred = model.predict_proba(X_test)[:,1]
predictions = y_pred.round()

accuracy = metrics.accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0), "AUC: ", metrics.roc_auc_score(y_score=y_pred,y_true=y_test))

Accuracy: 69.69% AUC:  0.7137681159420289


# Zad
Wykonaj Walidację krzyżową

In [47]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', MLPClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__hidden_layer_sizes': [(20,10)],
            'classifier__learning_rate_init': [0.001],#, 0.01, 0.1],
            'classifier__max_iter': [50],
            'classifier__batch_size': [8, 16],
}

grid_14 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_14.fit(X_train, y_train)
grid_14.best_params_

{'classifier__batch_size': 16,
 'classifier__hidden_layer_sizes': (20, 10),
 'classifier__learning_rate_init': 0.001,
 'classifier__max_iter': 50,
 'preprocessing': StandardScaler()}

In [48]:
metrics.accuracy_score(y_test, grid_2.best_estimator_.predict(X_test))

0.7834645669291339

In [50]:
from sklearn import  metrics


models = []
models.append(('LR', grid_1.best_estimator_))
models.append(('SVM linear', grid_2.best_estimator_))
models.append(('SVM rbf', grid_3.best_estimator_))
models.append(('KNN', grid_4.best_estimator_))
models.append(('DecisionTreeClassifier', grid_5.best_estimator_))
models.append(('BaggingClassifier', grid_6.best_estimator_))
models.append(('RandomForestClassifier', grid_7.best_estimator_))
models.append(('ExtraTreesClassifier', grid_8.best_estimator_))
models.append(('AdaBoostClassifier', grid_9.best_estimator_))
models.append(('GradientBoostingClassifier', grid_10.best_estimator_))
models.append(('XGBClassifier', grid_11.best_estimator_))
models.append(('voting_clf', voting_clf))
models.append(('MLP', grid_14.best_estimator_))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    
    if (name == 'SVM linear'):
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.decision_function(X_test)) ))            
    else:
        print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))
    if (name == 'SVM linear'):
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.decision_function(X_test)))        
    else:    
        roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))

LR
precision_score: 0.7435897435897436
recall_score: 0.6304347826086957
f1_score: 0.6823529411764706
accuracy_score: 0.7874015748031497
roc_auc_score: 0.8311191626409018
SVM linear
precision_score: 0.7341772151898734
recall_score: 0.6304347826086957
f1_score: 0.6783625730994152
accuracy_score: 0.7834645669291339
roc_auc_score: 0.829777241009125
SVM rbf
precision_score: 0.7236842105263158
recall_score: 0.5978260869565217
f1_score: 0.6547619047619048
accuracy_score: 0.7716535433070866
roc_auc_score: 0.8391706924315621
KNN
precision_score: 0.7121212121212122
recall_score: 0.5108695652173914
f1_score: 0.5949367088607596
accuracy_score: 0.7480314960629921
roc_auc_score: 0.7864667203435319
DecisionTreeClassifier
precision_score: 0.6666666666666666
recall_score: 0.5652173913043478
f1_score: 0.6117647058823529
accuracy_score: 0.7401574803149606
roc_auc_score: 0.8119296833064948
BaggingClassifier
precision_score: 0.7037037037037037
recall_score: 0.6195652173913043
f1_score: 0.6589595375722543
a

In [52]:
import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['SVM linear','SVM rbf','LR','KNN', 'DecisionTreeClassifier','BaggingClassifier','RandomForestClassifier','ExtraTreesClassifier', 'AdaBoostClassifier','GradientBoostingClassifier','XGBClassifier', 'voting', 'MLP'])
df

,Method,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
0,SVM linear,0.743590,0.630435,0.682353,0.787402,0.831119
1,SVM rbf,0.734177,0.630435,0.678363,0.783465,0.829777
2,LR,0.723684,0.597826,0.654762,0.771654,0.839171
3,KNN,0.712121,0.510870,0.594937,0.748031,0.786467
4,DecisionTreeClassifier,0.666667,0.565217,0.611765,0.740157,0.811930
5,BaggingClassifier,0.703704,0.619565,0.658960,0.767717,0.831186
6,RandomForestClassifier,0.670732,0.597826,0.632184,0.748031,0.817666
7,ExtraTreesClassifier,0.671642,0.489130,0.566038,0.728346,0.800959
8,AdaBoostClassifier,0.686747,0.619565,0.651429,0.759843,0.813137
9,GradientBoostingClassifier,0.677778,0.663043,0.670330,0.763780,0.816928


# Wczytaj dane treningowe i testowe

In [53]:
# Wczytaj dane treningowe i testowe

import pandas as pd

train_set = pd.read_csv('Dane/adult/adult.data', sep=", ",header = None)
test_set = pd.read_csv('Dane/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()



dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# print(X_train.shape)
# X_test.shape

In [54]:
print(X_train.shape)
print(X_test.shape)

(30162, 41)
(15060, 41)


# Zad
Porównaj wyniki sieci na:
* oryginalnych danych 
* na wystandaryzowanych

In [55]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', MLPClassifier())])

param_grid = {
            'preprocessing': [StandardScaler(), None],
            'classifier__hidden_layer_sizes': [(20,10)],
            'classifier__learning_rate_init': [0.001],
            'classifier__max_iter': [50],
            'classifier__batch_size': [16],
}

grid_15 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_15.fit(X_train, y_train)
grid_15.best_params_

{'classifier__batch_size': 16,
 'classifier__hidden_layer_sizes': (20, 10),
 'classifier__learning_rate_init': 0.001,
 'classifier__max_iter': 50,
 'preprocessing': StandardScaler()}

In [56]:
pipe = Pipeline([('classifier', MLPClassifier())])

param_grid = {
            'classifier__hidden_layer_sizes': [(20,10)],
            'classifier__learning_rate_init': [0.001],
            'classifier__max_iter': [50],
            'classifier__batch_size': [16],
}

grid_16 = GridSearchCV(pipe, param_grid, cv=kfold, return_train_score=True)

grid_16.fit(X_train, y_train)
grid_16.best_params_

{'classifier__batch_size': 16,
 'classifier__hidden_layer_sizes': (20, 10),
 'classifier__learning_rate_init': 0.001,
 'classifier__max_iter': 50}

In [58]:
from sklearn import  metrics


models = []
models.append(('MLP Scaled', grid_15.best_estimator_))
models.append(('MLP', grid_16.best_estimator_))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
roc_auc_score = []
for name, model in models:
    print(name)
    print("precision_score: {}".format(metrics.precision_score(y_test , model.predict(X_test)) ))
    print("recall_score: {}".format( metrics.recall_score(y_test , model.predict(X_test)) ))
    print("f1_score: {}".format( metrics.f1_score(y_test , model.predict(X_test)) ))
    print("accuracy_score: {}".format( metrics.accuracy_score(y_test , model.predict(X_test)) ))
    print("roc_auc_score: {}".format( metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]) ))
    
    precision_score.append(metrics.precision_score(y_test , model.predict(X_test)))
    recall_score.append(metrics.recall_score(y_test , model.predict(X_test)))
    f1_score.append( metrics.f1_score(y_test , model.predict(X_test)))
    accuracy_score.append(metrics.accuracy_score(y_test , model.predict(X_test)))    
    roc_auc_score.append(metrics.roc_auc_score(y_test , model.predict_proba(X_test)[:,1]))

import pandas as pd
d = {'precision_score': precision_score, 
     'recall_score': recall_score, 
     'f1_score': f1_score,
     'accuracy_score' : accuracy_score,
     'roc_auc_score' : roc_auc_score
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=['MLP Scaled', 'MLP'])
df

MLP Scaled
precision_score: 0.71760391198044
recall_score: 0.6345945945945946
f1_score: 0.6735513482501434
accuracy_score: 0.848871181938911
roc_auc_score: 0.90587652264941
MLP
precision_score: 0.6798882681564246
recall_score: 0.6578378378378379
f1_score: 0.6686813186813186
accuracy_score: 0.8398406374501992
roc_auc_score: 0.8991870003806623


,Method,precision_score,recall_score,f1_score,accuracy_score,roc_auc_score
0,MLP Scaled,0.717604,0.634595,0.673551,0.848871,0.905877
1,MLP,0.679888,0.657838,0.668681,0.839841,0.899187
